In [53]:
import os, sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchtext 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import classification_report
from sklearn import svm, model_selection, preprocessing, CountVectorizer

In [26]:
dataFile = '../Data/Reddit_News_DJIA_clean2.csv'

In [27]:
# Combine news titles into 1 feature
def transform(sample):
    return {'label': sample['label'], 'features': ' '.join(np.array(sample['features']))} 

In [29]:
parentPath = os.path.abspath('.')
sys.path.append(parentPath)
from RedditDJIA import RedditNewsDJIADataset

train_newsDataFrame = RedditNewsDJIADataset(dataFile, train = True, transform=transform)
for i, row in enumerate(train_newsDataFrame):
    #print(row)
    print(i, "Label =", row['label'], "Features =", row['features'])

    if i == 3:
        break

test_newsDataFrame = RedditNewsDJIADataset(dataFile, train = False, transform=transform)



0 Label = 0 Features = georgia downs two russian warplanes countries move brink war breaking musharraf impeached russia today columns troops roll south ossetia footage fighting youtube russian tanks moving towards capital south ossetia reportedly completely destroyed georgian artillery fire afghan children raped impunity n official says sick three year old raped nothing 150 russian tanks entered south ossetia whilst georgia shoots two russian jets breaking georgia invades south ossetia russia warned would intervene side enemy combatent trials nothing sham salim haman sentenced 5 1 2 years kept longer anyway feel like georgian troops retreat osettain capital presumably leaving several hundred people killed video prep georgia war russia rice gives green light israel attack iran says veto israeli military ops announcing class action lawsuit behalf american public fbi russia georgia war nyt top story opening ceremonies olympics fucking disgrace yet proof decline journalism china tells bush

In [37]:
# return row 0, combined features
train_newsDataFrame[0]['features']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# how to get features from each row
for row in train_newsDataFrame:
    #print(row['features'])
    pass

In [38]:

train_data = []
train_labels = []
for row in train_newsDataFrame:
    train_data.append(row['features'])
    train_labels.append(row['label'])

test_data = [] 
test_labels = [] 
for row in test_newsDataFrame:
    test_data.append(row['features']) 
    test_labels.append(row['label'])

In [49]:
# Create feature vectors 
vectorizer = TfidfVectorizer(min_df=5, max_df=0.9)
# Train the feature vectors
train_vectors = vectorizer.fit_transform(train_data)
# Apply model on test data 
test_vectors = vectorizer.transform(test_data)

In [46]:
train_vectors[0]

<1x31120 sparse matrix of type '<class 'numpy.float64'>'
	with 184 stored elements in Compressed Sparse Row format>

In [50]:
# Perform classification with SVM, kernel=linear 
model = svm.SVC(kernel='linear') 
model.fit(train_vectors, train_labels) 
prediction = model.predict(test_vectors)
print (classification_report(test_labels, prediction))

              precision    recall  f1-score   support

           0       0.44      0.26      0.33       196
           1       0.49      0.68      0.57       202

    accuracy                           0.47       398
   macro avg       0.46      0.47      0.45       398
weighted avg       0.46      0.47      0.45       398



In [54]:
trainDF = pd.DataFrame()
trainDF['text'] = test_data
trainDF['label'] = test_labels

In [55]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [57]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

Count Vectors

In [58]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

NameError: name 'CountVectorizer' is not defined